In [1]:
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
import os
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import joblib
from datetime import datetime

In [2]:
# Load the cleaned structured data
train_data_cleaned = pd.read_pickle('Fast_Furious_Insured/processed_data/train_data_cleaned.pkl')
test_data_cleaned = pd.read_pickle('Fast_Furious_Insured/processed_data/test_data_cleaned.pkl')

# Display the first few rows to confirm
print(train_data_cleaned.head())
print(test_data_cleaned.head())

print(train_data_cleaned.shape)
print(test_data_cleaned.shape)

print(train_data_cleaned.dtypes)
print(test_data_cleaned.dtypes)

                                          Image_path Insurance_company  \
0  Fast_Furious_Insured/images/train_images/img_4...                BQ   
1  Fast_Furious_Insured/images/train_images/img_7...                BQ   
2  Fast_Furious_Insured/images/train_images/img_4...                 A   
3  Fast_Furious_Insured/images/train_images/img_7...                 A   
4  Fast_Furious_Insured/images/train_images/img_7...                AC   

   Cost_of_vehicle  Min_coverage Expiry_date  Max_coverage  Condition  Amount  
0          41500.0        1037.5  03-12-2026      36142.68          0     0.0  
1          50700.0        1267.5  10-07-2025      12753.00          1  6194.0  
2          49500.0        1237.5  11-08-2022      43102.68          0     0.0  
3          33500.0         837.5  02-08-2022       8453.00          1  7699.0  
4          27600.0         690.0  01-05-2026       6978.00          1  8849.0  
                                          Image_path Insurance_company  \
0

In [3]:
print(train_data_cleaned['Expiry_date'].isnull().sum())  # To check if there are any missing paths

0


In [4]:
print(test_data_cleaned['Expiry_date'].isnull().sum())  # To check if there are any missing paths

0


In [5]:
print(train_data_cleaned['Insurance_company'].isnull().sum())  # Count missing values

0


In [6]:
print(train_data_cleaned['Insurance_company'].dtype)


object


In [7]:
from scipy.stats import skew

skewness = skew(train_data_cleaned['Amount'])
print(f"Skewness: {skewness}")

Skewness: 4.352416666529772


### Turning Expiry_Date from an object to numeric value

In [8]:
print(test_data_cleaned['Expiry_date'].dtype)  # Check column data type
print(test_data_cleaned['Expiry_date'].head())  # Inspect the first few values
print(train_data_cleaned['Expiry_date'].dtype)  # Check column data type
print(train_data_cleaned['Expiry_date'].head())  # Inspect the first few values

object
0    12-04-2025
1    24-08-2028
2    28-11-2023
3    04-02-2028
4    03-01-2022
Name: Expiry_date, dtype: object
object
0    03-12-2026
1    10-07-2025
2    11-08-2022
3    02-08-2022
4    01-05-2026
Name: Expiry_date, dtype: object


In [9]:
def convert_expiry_date(df, date_column='Expiry_date'):
    """
    Converts the 'Expiry_date' column from object to string, then standardizes the format to DD-MM-YYYY,
    converts it to datetime, and finally converts it to numeric (epoch time).

    Parameters:
    df (DataFrame): The dataframe containing the date column.
    date_column (str): The column name containing the expiry date.

    Returns:
    DataFrame: The dataframe with the converted expiry date in numeric format (epoch time).
    """

    # Step 1: Ensure the column is treated as a string
    df[date_column] = df[date_column].astype(str)

    # Step 2: Detect the date format (YYYY-MM-DD or DD-MM-YYYY)
    sample_date = df[date_column].dropna().iloc[0]  # Get a non-null sample date
    
    if '-' in sample_date:
        date_parts = sample_date.split('-')
        if len(date_parts[0]) == 4:  # YYYY-MM-DD format detected
            print("Detected YYYY-MM-DD format. Converting to DD-MM-YYYY.")
            df[date_column] = pd.to_datetime(df[date_column], format='%Y-%m-%d', errors='coerce').dt.strftime('%d-%m-%Y')
        else:
            print("Detected DD-MM-YYYY format. No conversion needed.")

    # Step 3: Convert to datetime format
    df[date_column] = pd.to_datetime(df[date_column], format='%d-%m-%Y', errors='coerce')

    # Step 4: Convert datetime to numeric (epoch time)
    df[date_column] = df[date_column].astype('int64') // 10**9  # Convert to Unix timestamp (seconds)

    print(f"Converted {date_column} to epoch time successfully.")
    return df


In [10]:
train_data_cleaned = convert_expiry_date(train_data_cleaned)
test_data_cleaned = convert_expiry_date(test_data_cleaned)
print(train_data_cleaned['Expiry_date'])
print(test_data_cleaned['Expiry_date'])

Detected DD-MM-YYYY format. No conversion needed.
Converted Expiry_date to epoch time successfully.
Detected DD-MM-YYYY format. No conversion needed.
Converted Expiry_date to epoch time successfully.
0       1796256000
1       1752105600
2       1660176000
3       1659398400
4       1777593600
           ...    
1305    1641513600
1306    1739750400
1307    1703894400
1308    1669334400
1309    1665360000
Name: Expiry_date, Length: 1310, dtype: int64
0      1744416000
1      1850688000
2      1701129600
3      1833235200
4      1641168000
          ...    
595    1729641600
596    1740096000
597    1689206400
598    1714867200
599    1754524800
Name: Expiry_date, Length: 600, dtype: int64


In [11]:
# Check rows where Expiry_date is invalid
invalid_dates = train_data_cleaned[train_data_cleaned['Expiry_date'].isna()]
print(invalid_dates[['Expiry_date']])

Empty DataFrame
Columns: [Expiry_date]
Index: []


In [12]:
print(train_data_cleaned.dtypes)
print(test_data_cleaned.dtypes)

Image_path            object
Insurance_company     object
Cost_of_vehicle      float64
Min_coverage         float64
Expiry_date            int64
Max_coverage         float64
Condition              int64
Amount               float64
dtype: object
Image_path            object
Insurance_company     object
Cost_of_vehicle        int64
Min_coverage         float64
Expiry_date            int64
Max_coverage         float64
dtype: object


In [13]:
print(train_data_cleaned['Expiry_date'].isnull().sum())  # To check if there are any missing paths

0


In [14]:
print(test_data_cleaned['Expiry_date'].isnull().sum())  # To check if there are any missing paths

0


In [15]:
print(train_data_cleaned['Expiry_date'])

0       1796256000
1       1752105600
2       1660176000
3       1659398400
4       1777593600
           ...    
1305    1641513600
1306    1739750400
1307    1703894400
1308    1669334400
1309    1665360000
Name: Expiry_date, Length: 1310, dtype: int64


### Feature Scaling

In [16]:
# Define feature columns (excluding 'Amount' initially)
feature_columns = ['Cost_of_vehicle', 'Min_coverage', 'Max_coverage', 'Expiry_date']

# Initialize the scaler for features
feature_scaler = StandardScaler()

# Fit and transform feature columns
train_data_cleaned[feature_columns] = feature_scaler.fit_transform(train_data_cleaned[feature_columns])

# Apply log transformation before scaling
train_data_cleaned['Amount'] = np.log1p(train_data_cleaned['Amount'])  # log(Amount + 1)

# Scale 'Amount' separately using another scaler
amount_scaler = StandardScaler()
train_data_cleaned['Amount'] = amount_scaler.fit_transform(train_data_cleaned[['Amount']])

# Save the scalers for later use
joblib.dump(feature_scaler, 'feature_scaler.pkl')
joblib.dump(amount_scaler, 'amount_scaler.pkl')
print(train_data_cleaned.shape)
print(train_data_cleaned.tail())

(1310, 8)
                                             Image_path Insurance_company  \
1305  Fast_Furious_Insured/images/train_images/img_4...                AC   
1306  Fast_Furious_Insured/images/train_images/img_4...                DA   
1307  Fast_Furious_Insured/images/train_images/img_4...                BQ   
1308  Fast_Furious_Insured/images/train_images/img_4...                AA   
1309  Fast_Furious_Insured/images/train_images/img_4...                 A   

      Cost_of_vehicle  Min_coverage  Expiry_date  Max_coverage  Condition  \
1305         1.574982      1.574982    -1.611635      0.238804          1   
1306         1.664688      1.664688    -0.147302      0.266733          1   
1307         0.453657      0.453657    -0.681777     -0.110309          1   
1308        -0.678881     -0.678881    -1.196934     -0.462914          1   
1309        -0.477042     -0.477042    -1.256177     -0.400073          1   

        Amount  
1305  0.356944  
1306  0.408026  
1307  0.27543

In [17]:
# Apply the same transformation to test data (except 'Amount', which is not available)
test_data_cleaned[feature_columns] = feature_scaler.transform(test_data_cleaned[feature_columns])
print(test_data_cleaned.tail())

                                            Image_path Insurance_company  \
595  Fast_Furious_Insured/images/test_images/img_77...                 B   
596  Fast_Furious_Insured/images/test_images/img_45...                 O   
597  Fast_Furious_Insured/images/test_images/img_45...                BQ   
598  Fast_Furious_Insured/images/test_images/img_45...                AA   
599  Fast_Furious_Insured/images/test_images/img_46...                RE   

     Cost_of_vehicle  Min_coverage  Expiry_date  Max_coverage  
595        -0.734947     -0.734947    -0.297986     -0.480369  
596         1.552556      1.552556    -0.142151      0.231822  
597        -1.172264     -1.172264    -0.900719      1.715508  
598         0.577003      0.577003    -0.518215     -0.071907  
599         1.328291      1.328291     0.072927      0.161999  


### One-Hot Encoding Insurance company

In [18]:
# Initialize the encoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='error')

# Fit the encoder on the training data and transform the 'Insurance_company' column in the train dataset
train_data_encoded = encoder.fit_transform(train_data_cleaned[['Insurance_company']])

In [19]:
# Convert the encoded data back to a DataFrame
train_data_encoded = pd.DataFrame(train_data_encoded, columns=encoder.get_feature_names_out(['Insurance_company']))

In [21]:
import pickle
with open(r"C:\Users\varsh\OneDrive\Desktop\notebook\Fast_Furious_Insured\encoder.pkl", "wb") as f:
    pickle.dump(encoder, f)

In [ ]:
print(train_data_encoded.shape)
print(train_data_encoded.tail())

In [ ]:
# Drop the original 'Insurance_company' column from train_data_cleaned
train_data_cleaned = train_data_cleaned.drop(columns=['Insurance_company'])

In [ ]:
# Join the one-hot encoded columns with the original DataFrame
train_data_cleaned = train_data_cleaned.join(train_data_encoded)

In [ ]:
print(train_data_cleaned.shape)
print(train_data_cleaned.tail())

In [ ]:
# Transform the 'Insurance_company' column in the test dataset using the same encoder
test_data_encoded = encoder.transform(test_data_cleaned[['Insurance_company']])

In [ ]:
# Convert the transformed data back to a DataFrame
test_data_encoded = pd.DataFrame(test_data_encoded, columns=encoder.get_feature_names_out(['Insurance_company']))

In [ ]:
# Drop the original 'Insurance_company' column from test_data_cleaned
test_data_cleaned = test_data_cleaned.drop(columns=['Insurance_company'])

In [ ]:
# Join the one-hot encoded columns with the test dataset
test_data_cleaned = test_data_cleaned.join(test_data_encoded)

In [ ]:
print(train_data_cleaned.shape)
print(test_data_cleaned.shape)

In [ ]:
print(train_data_cleaned.tail())
print(test_data_cleaned.tail())

In [ ]:
print(train_data_cleaned['Insurance_company_A'].isnull().sum())

### Image Preprocessing

In [ ]:
# import os

# # Folder containing all images
# image_folder = "Fast_Furious_Insured/images/train_images"

# # Get the list of images that are still relevant
# valid_images = set(train_data_cleaned["Image_path"].apply(os.path.basename))  # Extract filenames only

# # List all files in the folder
# all_images = set(os.listdir(image_folder))

# # Find extra images that need to be deleted
# extra_images = all_images - valid_images

# # Delete extra images
# for img in extra_images:
#     img_path = os.path.join(image_folder, img)
#     os.remove(img_path)  # Deletes the file
#     print(f"Deleted: {img_path}")

# print(f"✅ Removed {len(extra_images)} unnecessary images.")


In [ ]:
#No. of Images
train_images_path = "Fast_Furious_Insured/images/train_images"
test_images_path = "Fast_Furious_Insured/images/test_images"

num_train_images = len(os.listdir(train_images_path))
num_test_images = len(os.listdir(test_images_path))

print(f"Number of training images: {num_train_images}")
print(f"Number of test images: {num_test_images}")

In [ ]:
IMG_SIZE = (224, 224)  # ResNet50 requires images to be 224x224

def preprocess_image(image_path):
    # Load the image using OpenCV
    image = cv2.imread(image_path)
    
    # Resize the image to the target size (224x224 for ResNet50)
    image_resized = cv2.resize(image, IMG_SIZE)
    
    # Convert image to float32 for normalization
    image_normalized = image_resized.astype('float32') / 255.0
    
    # Preprocess the image using ResNet50 preprocessing (mean subtraction, etc.)
    image_processed = preprocess_input(image_normalized)
    
    return image_processed


In [ ]:
# Load the pre-trained ResNet50 model (exclude top layers for feature extraction)
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def extract_image_features(image_path):
    # Preprocess the image
    image_processed = preprocess_image(image_path)
    
    # Expand dimensions for batch processing (ResNet50 expects a batch)
    image_batch = np.expand_dims(image_processed, axis=0)
    
    # Extract features using the ResNet50 model
    features = base_model.predict(image_batch)
    
    # Flatten the feature vector
    return features.flatten()


In [ ]:
def combine_features(image_paths, structured_data):
    image_features = []
    
    # Iterate over all image paths and extract features
    for path in image_paths:
        features = extract_image_features(path)
        image_features.append(features)
    
    # Convert image features into a DataFrame
    image_features_df = pd.DataFrame(image_features, index=structured_data.index)

     # Print to check before merging
    print("Image Features Shape:", image_features_df.shape)
    print("Structured Data Shape:", structured_data.shape)
    
    # Concatenate image features with structured data (merge on index)
    combined_data = structured_data.join(image_features_df)
    
    return combined_data


In [ ]:
# Get the image paths from the 'Image_Path' column
train_image_paths = train_data_cleaned['Image_path'].tolist()
test_image_paths = test_data_cleaned['Image_path'].tolist()

In [ ]:
# Combine image features with the structured data
train_combined = combine_features(train_image_paths, train_data_cleaned)



In [ ]:
test_combined = combine_features(test_image_paths, test_data_cleaned)

In [ ]:
train_combined.shape

In [ ]:
test_combined.shape

In [ ]:
# # Extract image filenames from DataFrame
# train_images_used = set(train_data_cleaned["Image_path"].apply(lambda x: x.split("/")[-1]))

# # List actual image files in the folder
# import os
# image_folder = "Fast_Furious_Insured/images/train_images"
# all_image_files = set(os.listdir(image_folder))

# # Find extra images that are not in train_data_cleaned
# extra_images = all_image_files - train_images_used

# print(f"Extra images processed: {len(extra_images)}")
# print(f"Sample extra images: {list(extra_images)[:5]}")

In [ ]:
# import os

# # Get the list of image filenames in the folder
# image_folder = "Fast_Furious_Insured/images/train_images"
# all_image_files = set(os.listdir(image_folder))

# # Extract filenames from the train dataset
# train_image_filenames = set(train_data_cleaned["Image_path"].apply(lambda x: os.path.basename(x)))

# # Identify extra images (those in the folder but not in the dataset)
# extra_images = all_image_files - train_image_filenames
# missing_images = train_image_filenames - all_image_files

# print(f"Extra images in the folder: {len(extra_images)}")
# print(f"Missing images: {len(missing_images)}")


In [ ]:
# # Filter out rows where the image file doesn't exist
# valid_image_paths = set(train_data_cleaned["Image_path"].apply(lambda x: os.path.basename(x)))
# all_images_in_folder = set(os.listdir(image_folder))

# # Keep only records with matching image files
# train_data_cleaned_filtered = train_data_cleaned[train_data_cleaned["Image_path"].apply(
#     lambda x: os.path.basename(x) in all_images_in_folder
# )]

# print(f"Filtered train data shape: {train_data_cleaned_filtered.shape}")


In [ ]:
# Check the number of unique image paths in the cleaned dataset and processed data
print(f"Unique image paths in train_data_cleaned: {train_data_cleaned['Image_path'].nunique()}")
print(f"Unique image paths in processed train_combined: {train_combined['Image_path'].nunique()}")


In [ ]:
print(test_combined['Image_path'].isnull().sum())  # To check if there are any missing paths


In [ ]:
# Check for duplicates based on all columns in the combined dataset
duplicates = train_combined[train_combined.duplicated(subset='Image_path', keep=False)]  # Keep=False to mark all duplicates
print(duplicates)


In [ ]:
# Get the indexes of duplicated rows
duplicate_indexes = train_combined[train_combined.duplicated(subset='Image_path',keep=False)].index
print("Duplicate indexes:", duplicate_indexes)

In [ ]:
print(f"Length of duplicate_indexes: {len(duplicate_indexes)}")
print(f"Length of train_image_paths: {len(train_image_paths)}")
print(f"Shape of train_combined: {train_combined.shape}")

In [ ]:
train_combined.to_csv('Fast_Furious_Insured/processed_data/train_combined.csv', index=False)
test_combined.to_csv('Fast_Furious_Insured/processed_data/test_combined.csv', index=False)

In [ ]:
test_combined.shape

In [ ]:
# Save train_combined as pickle file
train_combined.to_pickle('Fast_Furious_Insured/processed_data/final_train_data.pkl')

# Save test_combined as pickle file
test_combined.to_pickle('Fast_Furious_Insured/processed_data/final_test_data.pkl')

print("Pickle files saved successfully!")